In [ ]:
# Imports
import pandas as pd
import os
import numpy as np

In [ ]:
# Some constants values
data_dir: str = '../dataset/CS2_36-38'
cleaned = '../dataset/cleaned'
df = pd.DataFrame()
hi = pd.DataFrame()

current = 'Current(A)'
voltage = 'Voltage(V)'
c_index = 'Cycle_Index'
# time = 'Test_Time(s)'
min_voltage = 3.8
max_voltage = 4.2001

datasets = {}

In [ ]:
#Cycle all excel files in the same folder
for filename in os.listdir(data_dir):
    xls_file = pd.ExcelFile(f'{data_dir}/{filename}')
    sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]
    
    df_list = []  # List to accumulate DataFrames for each sheet
    
    for sheet_fname in sheet_fnames:
        df_temp = pd.read_excel(f'{data_dir}/{filename}', sheet_name=sheet_fname)
        df_temp = df_temp[[voltage, current, c_index  ]]
        df_temp = df_temp[(df_temp[voltage] >= min_voltage) & (df_temp[voltage] <= max_voltage)]
        df_temp = df_temp[df_temp[voltage].diff().gt(0)]
        
        df_list.append(df_temp)  # Append the DataFrame to the list
    
    datasets[filename] = pd.concat(df_list).to_numpy()

# Now, concatenate the values from all datasets
df_cycle = np.concatenate([dataset[:, 0] for dataset in datasets.values()])
df_values = np.concatenate([np.delete(dataset, 0, axis=1) for dataset in datasets.values()])

# Calculate mean and std
X_mean, X_std = df_values.mean(0), df_values.std(0)

# Normalize values
df_values = (df_values - X_mean) / X_std

# Min and max after normalization
X_min, X_max = df_values.min(), df_values.max()

In [ ]:
ds_normalized = np.column_stack((df_values, df_cycle.reshape(-1, 1)))


In [ ]:
ds_normalized

In [ ]:
max_index = int(np.max(ds_normalized[:, 2]))

# Inizializza una lista per contenere i risultati
hi_data = []

# Inizializza hi_v e hi_i
hi_v = 0
hi_i = 0
index = 0
# Loop for per sommare gli elementi con il valore specifico nel terzo elemento
for element in ds_normalized:
    if element[2] == index:
        hi_v += element[0]
        hi_i += element[1]
    else:
        row = {'hi_v': hi_v, 'hi_i': hi_i}
        hi_data.append(row)
        index += 1
        hi_v = element[0]
        hi_i = element[1]

# Aggiungi l'ultima riga fuori dal ciclo
row = {'hi_v': hi_v, 'hi_i': hi_i}
hi_data.append(row)

# Crea il DataFrame finale
hi = pd.DataFrame(hi_data)


In [ ]:
hi.to_csv(f'{cleaned}/hi.csv')

In [ ]:
max_index = np.max(ds_normalized[:, 2])

# Inizializza una lista per contenere i risultati
hi_v = 0
hi_i = 0
index = 1
# Loop for per sommare gli elementi con il valore specifico nel terzo elemento
for element in ds_normalized:
    while index <= max_index:
        if element[2] == index:
            hi_v += element[0]
            hi_i += element[1]
        else: 
            row = pd.DataFrame([{'hi_v': hi_v, 'hi_i':hi_i}])
            index += 1
            hi = pd.concat([hi, row])
        

# df = pd.concat([df, df_temp[0]])
    # if row[2] == specific_value:
    #     row_sum = np.sum(row[:2])  # Somma i primi due elementi della riga
    #     sum_result.append(row_sum)

In [ ]:


# Creazione del DataFrame df_normalized
df_normalized = pd.DataFrame(data=ds_normalized, columns=['Voltage(V)', 'Current(A)', 'Cycle_Index'])

# Creazione delle colonne hi_v e hi_i
df_normalized['hi_v'] = df_normalized.groupby('Cycle_Index')['Voltage(V)'].cumsum()
df_normalized['hi_i'] = df_normalized.groupby('Cycle_Index')['Current(A)'].cumsum()

# Creazione di un nuovo DataFrame contenente solo le colonne desiderate
df_result = df_normalized[['hi_v', 'hi_i']]

# Salvataggio del nuovo DataFrame in un file CSV
df_result.to_csv('output.csv', index=False)


In [ ]:
for filename in os.listdir(data_dir):
    xls_file = pd.ExcelFile(f'{data_dir}/{filename}')
    sheet_fnames = [sheet for sheet in xls_file.sheet_names if sheet.startswith('Channel')]
    for sheet_fname in sheet_fnames:
        df_temp = pd.read_excel(f'{data_dir}/{filename}', sheet_name=sheet_fname)
        df_temp = df_temp[[c_index, current, voltage]]
        df_temp = df_temp[(df_temp[voltage] >= min_voltage) & (df_temp[voltage] <= max_voltage)]
        df_temp = df_temp[df_temp[voltage].diff().gt(0)]

    datasets[filename] = df_temp.to_numpy()



        # grouped_df = df_temp.groupby(df_temp[c_index])

        # for key, group in grouped_df:
        #     # voltage_normalized = (group[voltage] - min_voltage) / (max_voltage - min_voltage)
        #     hi_v = np.trapz(group[voltage])
        #     hi_i = np.trapz(group[current])
        #     row = pd.DataFrame([{'hi_v': hi_v, 'hi_i': hi_i}])
        #     hi = pd.concat([hi, row])
        #     print(hi)

        # max_index = df_temp[c_index].max()
        # df_temp = [df.reset_index(drop=True) for _, df in df.groupby(df_temp[c_index])]
        # for index in df_temp:
        #     hi_v = np.trapz(index[voltage]) 
        #     hi_i = np.trapz(index[current]) 
        #     row = pd.DataFrame([{'hi_v': hi_v, 'hi_i':hi_i}])
        #     hi = pd.concat([hi, row])

        # df = pd.concat([df, df_temp[0]])

In [ ]:
df_cycle = datasets[:, c_index]
df_values = np.delete(datasets, c_index, axis=1)

X_values = np.concatenate(list(df_values.values()))
X_mean, X_std = X_values.mean(), X_values.std()
X_values = (X_values - X_mean) / X_std
X_min, X_max = X_values.min(), X_values.max()

In [ ]:
ds_normalized = np.column_stack((X_values, c_index))

In [ ]:
datasets = df_temp.to_numpy()
datasets

In [ ]:
hi.to_csv(f'{cleaned}/hi.csv')

In [ ]:
for key, value in grouped_df:
    print(value) 

In [ ]:
print(df)
df.to_csv(f'{cleaned}/cleaned.csv')
